In [4]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

In [5]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals
from tensorly.decomposition import non_negative_parafac

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print('doing', k)
                # print(full)
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    return False


In [10]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 300
    a = np.random.randint(1, max, size=3) * 0.1
    b = np.random.randint(1, max, size=2)* 0.1
    c = np.random.randint(1, max, size=2)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

def low_tensor_inv():
    max = 300
    a = np.random.randint(1, max, size=2) * -0.1
    b = np.random.randint(1, max, size=3)* 0.1
    c = np.random.randint(1, max, size=3)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(2, 3, 3)) * 1.0
    return tens

In [11]:
t = low_tensor()+ low_tensor() + low_tensor()
t


array([[[10944.703,  5567.892],
        [ 8439.258,  5019.471]],

       [[21044.358, 10219.56 ],
        [15934.458,  9158.388]],

       [[30384.637, 13711.848],
        [22434.607, 11303.664]]])

In [245]:
check_rank(t,3, p = True, non_neg = True)

0.020181041542741883
[array([[1.32387342e+04, 4.96135996e+02, 1.66579248e+04],
       [9.33553674e+00, 1.03325000e+03, 3.08545931e+04]]), array([[0.38919243, 0.        , 0.5457797 ],
       [0.2998335 , 0.54673564, 0.47798031],
       [0.62226227, 0.41041383, 0.60154388]]), array([[0.57142245, 4.24041895, 0.13146303],
       [0.86766786, 7.23562994, 0.1245112 ],
       [1.96300277, 0.        , 1.16199264]])]
0.020182388360050485
[array([[1.87730579e+04, 7.05893394e+02, 1.05288830e+04],
       [1.31009318e+01, 1.47008418e+03, 1.95016331e+04]]), array([[0.25177745, 0.        , 0.64959829],
       [0.19396849, 1.6727428 , 0.5689018 ],
       [0.40255818, 1.25566564, 0.71597144]]), array([[0.62288705, 0.97413647, 0.17475511],
       [0.94581638, 1.66221609, 0.16551512],
       [2.13977369, 0.        , 1.54463649]])]
0.020188278965077605
[array([[5.65143861e+04, 1.03959579e+04, 1.62420193e+03],
       [3.56014268e+01, 1.92550058e+04, 3.38233579e+03]]), array([[0.27654243, 0.94170631, 0.    

False

In [217]:
B = P1.transpose() * P2.transpose()
B.eigenvects()

[(49.1666666666667,
  1,
  [Matrix([
   [ -0.917378869030917],
   [-0.0867790822056271],
   [ -0.388439701301379]])]),
 (1.05319148936148,
  1,
  [Matrix([
   [-0.828980717806854],
   [ -1.07995653145482],
   [-0.308015771295232]])]),
 (0.434944237918307,
  1,
  [Matrix([
   [-0.783472770444663],
   [  -1.0813219228451],
   [-0.369073949713589]])])]

In [182]:
C = P1 * P3
C.eigenvects()

[(14.6470588235294,
  1,
  [Matrix([
   [-0.379470952289146],
   [-0.917526780161815],
   [-0.118938656687643]])]),
 (0.931034482758621,
  1,
  [Matrix([
   [-0.143597906725126],
   [  -1.0105037880657],
   [-0.154234788704765]])]),
 (0.267857142857143,
  1,
  [Matrix([
   [-0.0974234014813838],
   [ -0.201805617354295],
   [ -0.285311390052624]])])]

In [210]:
t

array([[[16920.192 , 28519.892 , 20423.842 ],
        [ 6208.416 ,  8137.986 ,  5379.321 ],
        [ 2023.359 ,  4677.409 ,  2567.234 ]],

       [[15968.9336, 18838.7496, 14788.3776],
        [ 6340.1426,  6713.5946,  5204.4506],
        [ 1694.2368,  2401.3528,  1620.4508]]])